## LangSmith hub管理提示词
********
- 使用hub加载别人分享的提示词模板
- 使用langsmith sdk来做提示词管理

![](langsmith.png)

In [9]:
! pip install -qU langsmith -i https://pypi.tuna.tsinghua.edu.cn/simple


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


### 从hub上加载提示词
*****

- https://smith.langchain.com 注册并获取API KEY

In [3]:
import os
LANGCHAIN_API_KEY=os.environ.get("LANGCHAIN_API_KEY")

 - https://smith.langchain.com/hub 模板聚合广场

In [10]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


### 使用langsmith来管理提示词
*****
- 创建提示词
- 测试提示词
- 迭代提示词

- 创建提示词后上传到langsmith

In [11]:
from langsmith import Client
from langchain_core.prompts import ChatPromptTemplate


client = Client()

# Define the prompt

prompt = ChatPromptTemplate([
("system", "You are a helpful chatbot."),
("user", "{question}"),
])

# Push the prompt

client.push_prompt("test1-prompt", object=prompt)

'https://smith.langchain.com/prompts/test1-prompt/467cf1f5?organizationId=793cca2d-5ae8-539f-8476-90d5dd479c7f'

- 远程拉取自己的提示词并使用deepseek测试

In [12]:
from langsmith import Client
from langchain_deepseek import ChatDeepSeek
import os


client = Client()

llm = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-R1",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url=os.environ.get("DEEPSEEK_API_BASE"),
    )

# Pull the prompt to use

# You can also specify a specific commit by passing the commit hash "my-prompt:<commit-hash>"

prompt = client.pull_prompt("test1-prompt")

# Since our prompt only has one variable we could also pass in the value directly

# The code below is equivalent to formatted_prompt = prompt.invoke("What is the color of the sky?")

formatted_prompt = prompt.invoke({"question": "天空是什么颜色的?"})
print(formatted_prompt)
# Test the prompt

llm.invoke(formatted_prompt)

messages=[SystemMessage(content='You are a helpful chatbot.', additional_kwargs={}, response_metadata={}), HumanMessage(content='天空是什么颜色的?', additional_kwargs={}, response_metadata={})]


AIMessage(content='天空的颜色主要取决于时间和天气条件：\n\n1. **晴朗白天**：呈现蓝色，这是由于阳光中的短波蓝光被大气分子散射（瑞利散射现象）。\n\n2. **日出/日落**：常出现橙红色，因为阳光穿透更厚的大气层时蓝光被散射殆尽，剩下长波红光到达地面。\n\n3. **阴天**：灰白色，云层中的水滴散射所有波长的可见光。\n\n4. **夜晚**：呈现黑色，仅有星光和大气辉光点缀。\n\n特殊现象：\n- 极光出现时会有绿色/紫色光带\n- 沙尘暴天气可能呈现橙黄色\n- 月食期间可能出现"血月"的暗红色\n\n有趣的是，火星天空呈粉橙色，而金星天空是橙黄色，这都与不同行星的大气成分有关。地球的蓝色天空其实是大气层与阳光共同作用的独特光学效果。', additional_kwargs={'refusal': None, 'reasoning_content': '嗯，用户问“天空是什么颜色的？”，这个问题看起来简单，但其实可能有不同的情况需要考虑。首先，我得想想通常情况下的答案。在白天，晴朗的时候，天空是蓝色的，这是因为瑞利散射，阳光中的短波长光线（比如蓝色）更容易被大气中的分子散射，所以整个天空看起来是蓝色的。不过，如果是在日出或日落的时候，太阳的位置低，光线穿过更多的大气层，蓝光被散射掉更多，剩下的红光和黄光会让天空呈现橙色或红色。\n\n然后，用户可能是在不同的时间或天气条件下观察天空，所以需要考虑不同的情况。比如阴天的时候，云层比较厚，天空会显得灰蒙蒙的。夜晚的话，天空是黑色的，因为有星星和月亮的光，但大部分区域还是黑暗的。另外，如果是在其他星球上，比如火星，天空的颜色会因为大气成分不同而不同，但用户可能问的是地球上的情况。\n\n还要考虑用户的年龄和知识水平。如果是小孩子问的，可能需要用更简单的语言解释，比如“因为阳光和空气中的小颗粒让天空看起来是蓝色的”。如果是成年人，可能更愿意了解科学原理，比如瑞利散射和米氏散射的区别，或者不同天气现象对天空颜色的影响。\n\n另外，用户可能没有明确说明他们问的具体情境，比如是否在污染严重的城市，或者是否有特殊天气现象如沙尘暴、雾霾等，这些都会影响天空的颜色。这时候可能需要进一步询问用户的具体情况，或者给出一个全面的回答，涵盖各种可能性。\n\n也有可能用户的问题背后有更深层的意图，比如

- 团队迭代提示词

In [13]:
from langsmith import Client
from langchain_core.prompts import ChatPromptTemplate

# Connect to the LangSmith client

client = Client()

# Define the prompt to update

new_prompt = ChatPromptTemplate([
("system", "你是一个有趣的机器人. 使用中文回复."),
("user", "{question}"),
])

# Push the updated prompt making sure to use the correct prompt name

# 注意tags可以更方便进行版本管理,你也可以打成v0.0.1这样

client.push_prompt("test1-prompt", object=new_prompt, tags=["Chinese"])

'https://smith.langchain.com/prompts/test1-prompt/3aa53ff1?organizationId=793cca2d-5ae8-539f-8476-90d5dd479c7f'